## 1. Testing the most basic approach of concatenating all genes alignments in a single [XMFA](https://darlinglab.org/mauve/user-guide/files.html) (eXtented Multi-FastA) file

In [79]:
! sed -e '$s/$/\n=/' -s ../test/data/aligned_gene_sequences_clean/*.fas > core_gene_alignment.xmfa

3741.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Creating a test tree from one of the genes with rapidnj (approach is wrong but, just to get a tree)

In [80]:
! rapidnj -i fa  ../test/data/aligned_gene_sequences_clean/acpP.aln.fas > acpP_njtree.nwk

3750.53s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [81]:
! sed -i "s/'//g" acpP_njtree.nwk 

3758.74s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [82]:
! ClonalFrameML acpP_njtree.nwk core_gene_alignment.xmfa core_gene_test_cfml -xmfa_file true -show_progress true -output_filtered true

3765.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ClonalFrameML v1.12
xmfa_file = true
show_progress = true
output_filtered = true
Finished reading in control file.

Read 3 sequences of length 2987 sites from core_gene_alignment.xmfa
IMPUTATION AND RECONSTRUCTION OF ANCESTRAL STATES:
Analysing 4 sites
Empirical nucleotide frequencies:   A 32.6%   C 16.3%   G 22.9%   T 28.2%
Maximum log-likelihood for imputation and ancestral state reconstruction = -30.9854
Wrote imputed and reconstructed ancestral states to core_gene_test_cfml.ML_sequence.fasta
Wrote position cross-reference file to core_gene_test_cfml.position_cross_reference.txt
BRANCH LENGTH CORRECTION/RECOMBINATION ANALYSIS:
Analysing 987 sites
Empirical nucleotide frequencies:   A 32.6%   C 16.3%   G 22.9%   T 28.2%
Beginning branch optimization. Key to parameters (and constraints):
B   uncorrected branch length
L   maximum unnormalized log-posterior per branch
P   contribution of the prior to L
R   R/theta per branch                                       (> 0)
I   mean DNA impor

This simple approach has different drawbacks:
- ClonalFrameML will fail if genomes are missing a gene (if core genome definition is less than 100%)
- The XMFA format standard is not respected, and biopython will fail to read the file
- The order of the genes is the order of the files in the system
- If there are unsupported bases (other than `{A,T,G,C,N,-}`), ClonalFrameML will fail

In [72]:
import pysam
from pathlib import Path
import glob
from textwrap import wrap

In [93]:
def replace_ambiguous(seq, alphabet=set(['A','T','C','G','N','-'])):
    seq = seq.upper()
    all_chars = set(seq)
    replace_chars = all_chars - alphabet
    trans_table = str.maketrans({char: 'N' for char in replace_chars})
    return seq.translate(trans_table)

In [44]:
fas = [Path(p) for p in glob.glob('../test/data/aligned_gene_sequences_raw/*.fas')]

In [95]:
seqdict = {}
genomes = set()
gene_aln_len = {}
for gene in fas:
    gene_name = gene.name.split('.')[0]
    seqdict[gene_name] = {}
    for record in pysam.FastxFile(gene):
        gene_aln_len[gene_name] = len(record.sequence)
        recname = record.name.split(';')[0]
        genomes.add(recname)
        seqdict[gene_name][recname] = replace_ambiguous(record.sequence)
genomes = sorted(genomes)

In [96]:
gene_order = []
for record in pysam.FastxFile('../test/data/pan_genome_reference.fa'):
    gene_order.append(record.name)

In [97]:
seqdict

{'acpP': {'GCF_001640785': 'ATGACTAAAGAAGAGGTATTTACCAAGATTACCGATATCATTGTGGATCATTTTGATGTCGATAAAGCAAAAGTTACAAACGATTTGAACTTTAAAACTGATTTGAATGCAGATTCAATTGATATTGTTGAATTTGTTCTTGAATTGGAAGATACATTTGGCGCGGAAATTTCTGATGAGGATGCTGAAAAGCTTTCCACAGTCGGTGAAGCAGTTGATTTCATTACCGCCAATCAAAAGAAATAA---',
  'GCF_007990205': 'ATGACTAAAGAAGAGGTATTTACCAAGATTACCGATATCATTGTGGATCATTTTGATGTCGATAAAGCAAAAGTTACAAACGATTTGAACTTTAAAACTGATTTGAACGCAGATTCAATTGATATTGTTGAATTTGTTCTTGAATTGGAAGATACATTTGGCGCGGAAATTTCTGATGAGGATGCTGAAAAGCTTTCCACAGTCGGTGAAGCAGTTGATTTCATTACCGCCAATCAAAAGAAATAA---',
  'GCF_014651215': 'ATGACTAAAGAAGAGGTATTTACCAAGATTACCGATATCATTGTGGATCATTTTGATGTCGATAAAGCAAAAGTTACAAACGATTTGAACTTTAAAACTGATTTGAACGCAGATTCAATTGATATTGTTGAATTTGTTCTTGAATTGGAAGATACATTTGGCGCGGAAATTTCTGATGAGGATGCTGAAAAGCTTTCCACAGTCGGTGAAGCAGTTGATTTCATTACCGCCAATCAAAAGAAATAA---'},
 'atpE': {'GCF_001640785': 'ATGGGAGCTATTGCAGCAGGAATTATTATGTTCGGAGCCGCTATCGGTGGTGGTATCGGTGATGCCCTTGTTATTTCAAAAACACTTGAAGGAATGGCACGCCAACCAGAATTATCTGGTCAATTAAGAACAACTATGTTTATTGGT

In [98]:
with(open('core_gene_alignment_test.xmfa', 'w')) as fw:
    i = 0
    for gene in gene_order:
        if gene in seqdict:
            fw.write(f"# {gene}\n")
            for genome in genomes:
                if genome in seqdict[gene]:
                    fw.write(f">{genome}:{i}-{i+gene_aln_len[gene]}\n")
                    fw.write("\n".join(wrap(seqdict[gene][genome], 60))+"\n")
                        # seqdict[gene][genome]+"\n")
                else:
                    fw.write(f">{genome}\n")
                    fw.write("\n".join(wrap('-'*gene_aln_len[gene], 60))+"\n")
            i += gene_aln_len[gene]
            fw.write("=\n")

In [99]:
! ClonalFrameML acpP_njtree.nwk core_gene_alignment_test.xmfa core_gene_test_clean_cfml -xmfa_file true -show_progress true -output_filtered true

4083.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ClonalFrameML v1.12
xmfa_file = true
show_progress = true
output_filtered = true
Finished reading in control file.

Read 3 sequences of length 2987 sites from core_gene_alignment_test.xmfa
IMPUTATION AND RECONSTRUCTION OF ANCESTRAL STATES:
Analysing 4 sites
Empirical nucleotide frequencies:   A 33.2%   C 16.3%   G 22.7%   T 27.8%
Maximum log-likelihood for imputation and ancestral state reconstruction = -30.9981
Wrote imputed and reconstructed ancestral states to core_gene_test_clean_cfml.ML_sequence.fasta
Wrote position cross-reference file to core_gene_test_clean_cfml.position_cross_reference.txt
BRANCH LENGTH CORRECTION/RECOMBINATION ANALYSIS:
Analysing 987 sites
Empirical nucleotide frequencies:   A 33.2%   C 16.3%   G 22.7%   T 27.8%
Beginning branch optimization. Key to parameters (and constraints):
B   uncorrected branch length
L   maximum unnormalized log-posterior per branch
P   contribution of the prior to L
R   R/theta per branch                                       (> 0)
I